In [ ]:
import datetime
import os
from typing import Callable, Optional
import pandas as pd
from sklearn import preprocessing
import numpy as np
import torch
import kagglehub
from kagglehub import KaggleDatasetAdapter
from graph_tool.all import *
from torch_geometric.data import (
    Data,
    InMemoryDataset
)

# Load the dataset

In [ ]:
dataset_name = "LI-Small_Trans.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "ealtman2019/ibm-transactions-for-anti-money-laundering-aml",
  dataset_name
)

In [ ]:
## Uncomment the followings to free up space
# import shutil
# shutil.rmtree(os.path.expanduser("~/.cache/kagglehub/datasets/ealtman2019/"))

# Look at the data
Draw some plots...

The following table explains the meaning of each column present in the dataset

| **Colonna**            | **Descrizione**                                                                 |
|------------------------|----------------------------------------------------------------------------------|
| Timestamp              | Year/Month/Day Hour/Minute                                                      |
| From Bank              | Numeric code for bank where transaction originates                              |
| Account (From)         | Hexadecimal code for account where transaction originates                       |
| To Bank                | Numeric code for bank where transaction ends                                    |
| Account (To)           | Hexadecimal code for account where transaction ends action ends                 |
| Amount Received        | Monetary amount received from the source account (in currency units of the next column) |
| Receiving Currency     | Currency such as dollars, euros, etc of From account                            |
| Amount Paid            | Monetary amount paid (in currency units of the next column)                     |
| Payment Currency       | Currency such as dollars, euros, etc of From account                            |
| Payment Format         | How transaction was conducted, e.g. cheque, ACH, wire, credit cards, etc.       |
| Is Laundering          | 0/1 value with 1 = Transaction is Laundering, 0 = Not                            |

# Data Cleaning

In [ ]:
print(f"Dataset has {len(df):,.0f} rows")
df.head(5)

To avoid conflicts, let's rename the bank account columns to specify the source and recipient of the transaction

In [ ]:
df.rename(columns={'Account': 'Source Account'}, inplace=True)
df.rename(columns={'Account.1': 'Destination Account'}, inplace=True)

In [ ]:
# Check across all the columns of the DataFrame whether they contain empty strings or infinity values
df.map(lambda x: x == '' or x == float('inf') or x == float('-inf') or x == np.inf or x==-np.inf).any()

In [ ]:
# Check for missing values
for col in df.columns:
    print(f"Column '{col}' has {df[col].isna().sum()/df[col].count()*100:0.2f}% of missing values")

Take a closer look at the types

In [ ]:
for col in df.columns:
    print(f"'{col}' has '{df[col].dtype}' dtype")

In [ ]:
# Change the type of the column 'Timestamp' to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
print(f"The type of the column 'Timestamp' has been changed to '{df['Timestamp'].dtype}' dtype")

## Identifying and removing duplicates

In [ ]:
# List the duplicated rows, including the first occurrence
duplicated_rows_map = df.duplicated(keep=False)
print(f"There are {duplicated_rows_map.sum()} duplicated rows in the dataset, including the first occurrence.")
df[duplicated_rows_map].sort_values(by=['Timestamp', 'From Bank', 'Source Account', 'To Bank', 'Destination Account'])

In [ ]:
# Let's drop the duplicated rows
df.drop_duplicates(inplace=True)
print(f"After dropping the duplicated rows, the dataset has {df.shape[0]:,.0f} rows.")

## Look for inconsistent values

In [ ]:
# Now we look for negative values in the columns containing amount value
lower_zero = ((df['Amount Received'] < 0) | (df['Amount Paid'] < 0)).sum()
print(f"There are {lower_zero} rows with negative values in the columns 'Amount Received' or 'Amount Paid'")

In [ ]:
inconsistent_transactions = ((df['Amount Received'] != df['Amount Paid']) & (df['Receiving Currency'] == df['Payment Currency'])).sum()
print(f"There are {inconsistent_transactions} transactions where the amount received is not equal to the amount paid, but the currencies are the same.")

# Data preprocessing

In the data preprocessing, we perform below transformation:

- Transform the Timestamp with min max normalization and extract some features from it (hour, day of month, month and day of the week)
- Create unique ID for each account by adding bank code with account number.
- Create receiving_df with the information of receiving accounts, received amount and currency
- Create paying_df with the information of payer accounts, paid amount and currency
- Create a list of currency used among all transactions
- Label the 'Payment Format', 'Payment Currency', 'Receiving Currency' by classes with sklearn LabelEncoder

In [ ]:
def df_label_encoder(df, columns):
    le = preprocessing.LabelEncoder()
    for i in columns:
        df[i] = le.fit_transform(df[i].astype(str))
    return df
    
def preprocess(df) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, list[str]]:
    # Extract some features from the 'Timestamp'
    df['hour'] = df['Timestamp'].dt.hour
    df['day of month'] = df['Timestamp'].dt.day
    df['month'] = df['Timestamp'].dt.month
    df['weekday'] = df['Timestamp'].dt.weekday
    
    # Put the 'Is Laundering' as last column
    cols = df.columns.tolist()
    cols.remove('Is Laundering')
    idx = cols.index('weekday') + 1
    cols.insert(idx, 'Is Laundering')
    df = df[cols]
    
    df = df_label_encoder(df,['Payment Format', 'Payment Currency', 'Receiving Currency'])
    
    # Scale the Timestamp feature to a real-valued range between 0 and 1 using min-max normalization
    df['Timestamp'] = df['Timestamp'].apply(lambda x: x.value)
    df['Timestamp'] = (df['Timestamp']-df['Timestamp'].min())/(df['Timestamp'].max()-df['Timestamp'].min())

    df['Source Account'] = df['From Bank'].astype(str) + '_' + df['Source Account']
    df['Destination Account'] = df['To Bank'].astype(str) + '_' + df['Destination Account']
    df = df.sort_values(by=['Source Account'])
    receiving_df = df[['Destination Account', 'Amount Received', 'Receiving Currency']]
    paying_df = df[['Source Account', 'Amount Paid', 'Payment Currency']]
    currency_ls = sorted(df['Receiving Currency'].unique())

    return df, receiving_df, paying_df, currency_ls

In [ ]:
edges_df, receiving_df, paying_df, currency_ls = preprocess(df)
edges_df.head(5)

We want to extract all unique accounts from payer and receiver as node of our graph. It includes the unique account ID, Bank code and the label of 'Is Laundering'.
In this section, we consider both payer and receiver involved in a illicit transaction as suspicious accounts, we will label both accounts with ```'Is Laundering' == 1```.

In [ ]:
def get_nodes(df: pd.DataFrame) -> pd.DataFrame:
	ldf = df[['Source Account', 'From Bank']]
	rdf = df[['Destination Account', 'To Bank']] 

	# Get all illicit transactions
	suspicious = df[df['Is Laundering']==1]

	# Separate source and destination accounts involved in illicit transactions.
	source_df = suspicious[['Source Account', 'Is Laundering']].rename({'Source Account': 'Account'}, axis=1)
	destination_df = suspicious[['Destination Account', 'Is Laundering']].rename({'Destination Account': 'Account'}, axis=1)

	# Joint into a single DataFrame
	suspicious = pd.concat([source_df, destination_df], join='outer')

	# An account could be involved in several illicit transactions, so we drop duplicates
	suspicious = suspicious.drop_duplicates()

	# Merge the source and destination accounts with their respective banks
	ldf = ldf.rename({'Source Account': 'Account', 'From Bank': 'Bank'}, axis=1)
	rdf = rdf.rename({'Destination Account': 'Account', 'To Bank': 'Bank'}, axis=1)
	df = pd.concat([ldf, rdf], join='outer')
	df = df.drop_duplicates()

	df['Is Laundering'] = 0

	# Mark all the transactions of the accounts involved in illicit transactions as illicit
	df.set_index('Account', inplace=True)
	df.update(suspicious.set_index('Account'))
	return df.reset_index()

In [ ]:
nodes_df = get_nodes(edges_df)
nodes_df.head(5)

# Topological properties

In [ ]:
trans_graph = Graph(
    list(edges_df[['Source Account', 'Destination Account']].itertuples(index=False, name=None)), 
    hashed=True,
    directed=True
)

In [ ]:
#vb, eb = betweenness(trans_graph)

In [ ]:
u = GraphView(trans_graph)

In [ ]:
graph_draw(u, output="transaction-graph.pdf")